In [2]:
import os
import re

from pathlib import Path

In [2]:
# script to separate report (OK)
# script to clean 3 lines of introduction but there are some that only have 2 lines, so need to check for the 3rd line first word -> "pemeriksaan" (OK)

# script to check whether there is wrong report file in folder
# script to detect which "kesan" is the conclusion 

'''
26
TS Yang terhormat

Pemeriksaan radiografi pelvis proyeksi AP dengan hasil sebagai berikut :

Kedudukan tulang-tulang pelvis berubah
Tampak fraktur pada neck femur dextra dengan fragmen fraktur displaced ke cranial
Celah sendi coxae dan sakroiliaka kanan-kiri serta simfisis pubis dalam batas normal.
Jaringan lunak tak tampak lesi patologis.


Kesan:
Fraktur pada neck femur dextra

Terima kasih atas kerjasamanya,
55 pelvis
TS Yang terhormat

Pemeriksaan radiografi pelvis proyeksi AP dengan hasil sebagai berikut :

Kedudukan tulang-tulang femur dextra berubah
Tampak fraktur pada intertrochanter femur dextra dengan fragmen fraktur displaced minimal
Celah sendi coxae dan sakroiliaka kanan-kiri serta simfisis pubis dalam batas normal.
Jaringan lunak tak tampak lesi patologis.

Kesan:
Fraktur pada intertrochanter femur dextra

Terima kasih atas kerjasamanya,
56 pelvis
TS Yang terhormat

Pemeriksaan radiografi pelvis proyeksi AP dengan hasil sebagai berikut :

Kedudukan tulang-tulang pelvis baik.
Tak tampak fraktur, destruksi, dislokasi, maupun lesi litik/blastik lainnya.
Celah sendi coxae dan sakroiliaka kanan-kiri serta simfisis pubis dalam batas normal.
Jaringan lunak tak tampak lesi patologis.


Kesan:
Tak tampak kelainan radiologik.


Terima kasih atas kerjasamanya,

'''

'\n26\nTS Yang terhormat\n\nPemeriksaan radiografi pelvis proyeksi AP dengan hasil sebagai berikut :\n\nKedudukan tulang-tulang pelvis berubah\nTampak fraktur pada neck femur dextra dengan fragmen fraktur displaced ke cranial\nCelah sendi coxae dan sakroiliaka kanan-kiri serta simfisis pubis dalam batas normal.\nJaringan lunak tak tampak lesi patologis.\n\n\nKesan:\nFraktur pada neck femur dextra\n\nTerima kasih atas kerjasamanya,\n55 pelvis\nTS Yang terhormat\n\nPemeriksaan radiografi pelvis proyeksi AP dengan hasil sebagai berikut :\n\nKedudukan tulang-tulang femur dextra berubah\nTampak fraktur pada intertrochanter femur dextra dengan fragmen fraktur displaced minimal\nCelah sendi coxae dan sakroiliaka kanan-kiri serta simfisis pubis dalam batas normal.\nJaringan lunak tak tampak lesi patologis.\n\nKesan:\nFraktur pada intertrochanter femur dextra\n\nTerima kasih atas kerjasamanya,\n56 pelvis\nTS Yang terhormat\n\nPemeriksaan radiografi pelvis proyeksi AP dengan hasil sebagai beriku

In [3]:
def txt_file(filepath):
    if filepath.endswith('TOTAL.txt'):
        pass
    else:
        return filepath.lower().endswith('.txt')

def separate_report(text):
    report = {}
    current_report = []
    num = 1
    for sentence in text.split('\n'):
        for word in sentence.split():
            # get report number for dict key
            if word.isdigit():
                num = word
            # if word is a digit (means new report) and it is not the first digit (report)
            if word.isdigit() and word != text.split()[0]:
                # combine all the words into one report and add it to dict
                final_report = ' '.join(current_report)
                final_report = '\n'.join([f.lstrip(' ') for f in final_report.splitlines()])
                report[num] = final_report
                # clear list and append new digit (report)
                current_report = []
                current_report.append(word)
            # elif last word in report, add the final report to the dict
            elif word == text.split()[-1]:
                current_report.append(word)
                final_report = ' '.join(current_report)
                final_report = '\n'.join([f.lstrip(' ') for f in final_report.splitlines()])
                report[num] = final_report
            else:
                current_report.append(word)
        current_report.append('\n')
    # {report_num_1: text, report_num_2: text}
    return report

def makedir(path):
    Path(path).mkdir(parents=True, exist_ok=True)
    # change permission into read, write, execute
    Path(path).chmod(0o777)

def generate_separate_report(text, base_dir, filename):
    report = separate_report(text)
    # ex path: base_dir/body_part/series/filename
    # ex filename: body_part-series+reportnum ??
    body_part = Path(filename).parent.parent.name
    for key, val in report.items():
        base_new_filename = f'{body_part}-num{key}_series_1001'
        new_filename = f'{base_new_filename}.txt' # need to confirm number and series
        new_filepath = os.path.join(base_dir, body_part, base_new_filename, new_filename)
        makedir(os.path.join(base_dir, body_part, base_new_filename))
        # create each report in the respective folders
        with open(new_filepath, 'w') as w:
            w.write(val)

def remove_start_end(text):
    # remove empty lines, start and end sentences that bear no meaning
    sentences = [s for s in text.splitlines() if s]
    end_word = sentences[-1].split()[0].lower()
    start_word = sentences[2].split()[0].lower()
    if end_word in ['terima', 'btk']:
        start_index = 3 if start_word in ['pemeriksaan', 'foto'] else 2
        text = os.linesep.join(sentences[start_index:-1])
    else:
        start_index = 3 if start_word in ['pemeriksaan', 'foto'] else 2
        text = os.linesep.join(sentences[start_index:])
    return text

def remove_punctuations(text):
    punctuations = '''!()[]{};:'"\,<>?@#$%^&*~.''' # / is not removed for fraction, - is not removed for range (ex: l3-l4)
    new_text = ''
    for char in text:
        if char not in punctuations: 
            new_text += char
    return new_text

def cleaning(text, base_dir, filename):
    generate_separate_report(text, base_dir, filename)
    text = text.strip()
    text = text.lower()
    text = remove_start_end(text)
    text = remove_punctuations(text)
    return text

In [30]:
def file_mapping(text: str, file_list: dict):
    # will store word mapping to file. format: {word: [filename1, filename2, ...]}
    word_mapping = {}
    # split compiled text and make it into set
    text = set(text.split())
    # loop through set
    for word in text:
        for key, val in file_list.items():
            if word == 'vesicolithpreperitoneal':
                print(val.split())
            # if word in text value of each file dict
            if word in val.split() and (word != '-' or '/'):
                # add word as key and the filename as value
                if word not in word_mapping:
                    word_mapping[word] = [key]
                else:
                    word_mapping[word].append(key)             
    return word_mapping

def generate_wordlist2(word_mapping: dict, body_part: str, word_list_dir: str):
    makedir(word_list_dir)
    with open(os.path.join(word_list_dir, f'{body_part}_with_file.txt'), 'w') as w:
        # loop through dictionary, generate file with word and filename
        for key, val in word_mapping.items():
            w.write(key + '     ' + str(val) + '\n')

In [32]:
# directories
base_dir = os.path.join('..','datasets','radiology_clean')
word_list_dir = os.path.join('..', 'datasets_wordlist_with_file')

# body part grouping
body_part_group = [f.lower() for f in os.listdir(base_dir) if os.path.isdir(os.path.join(base_dir, f))]
print("body part group:", body_part_group, "\n")

# compilation report per body part
# format: {body_part: 'all report text for the body part'}
body_part_text = {}
# compilation of each filename with the text
# format: {filename: text, filename2: text}
file_text_mapping = {}
# store compiled text of each body part
compiled_text = ''

idx = 0
for root, dirs, files in os.walk(base_dir):
    for file in files:
        # get full filename
        filename = os.path.join(root, file)
        # get body part
        body_part = Path(filename).parent.parent.name
        
        # if txt file -> report
        if txt_file(filename):
            f = open(filename, "r", encoding="unicode_escape")
            text = f.read()
            text = cleaning(text, base_dir, filename)

            # if still the same body part
            if body_part == body_part_group[idx]:
                # append to compiled text that contains all txt of the body part
                # append to file text mapping
                compiled_text += text + ' '
                file_text_mapping[file] = text
            
            # if already different body part
            elif body_part != body_part_group[idx]:
                # generate wordlist by passing the compiled text
                word_mapping = file_mapping(compiled_text, file_text_mapping)
                generate_wordlist2(word_mapping, body_part_group[idx], word_list_dir)
                compiled_text = text
                 # clear compiled text and file text mapping dict, append with new body part, add ix
                file_text_mapping = {}
                file_text_mapping[file] = text
                idx += 1

    # if reach end of file
    if not dirs:
        word_mapping = file_mapping(compiled_text, file_text_mapping)
        generate_wordlist2(word_mapping, body_part_group[idx], word_list_dir)


body part group: ['abodemen_3_posisi', 'pelvis', 'test'] 

